Code to extract and save $\Delta F / F$ for whole making whole brain maps

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import glob
from pathlib import Path
import pickle
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from janelia_core.dataprocessing.dataset import ROIDataset
from janelia_core.utils.data_saving import append_ts

## Parameters go here

In [3]:
ps = dict()

# Location of excel file specifying where the data for each experiment is saved relative to the base folder
ps['data_loc_file'] = r'A:\projects\keller_vnc\data\experiment_data_locations.xlsx'

# Subfolder containing the dataset for each subject
ps['dataset_folder'] = 'extracted'

# Base folder where datasets are stored 
ps['dataset_base_folder'] =r'K:\\SV4'

# Data to calculate Delta F/F for in each dataset
ps['f_ts_str'] = 'f_5_25_25'
ps['bl_ts_str'] = 'bl_5_25_25'

ps['n_smps'] = 3 # Number of samples to take before and after a manipulation 

# Parameters for calculating dff
ps['background'] = 100
ps['ep'] = 20

# Location to save results
ps['save_folder'] = r'A:\projects\keller_vnc\results\whole_brain_stats'
ps['save_str'] = 'dff_5_25_25_with_ep'

## Read in excel file specifying location of datasets

In [4]:
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

data_locs = pd.read_excel(ps['data_loc_file'], header=1, usecols=[1, 2], converters=converters)

## Helper function to calculate $\Delta F/F$

In [5]:
def calc_dff(f, b, background=ps['background'], ep=ps['ep']):
    return (f-b)/(b-background+ep)

## Extract dff along with behavior information from each dataset

In [11]:
n_datasets = len(data_locs)
event_annots = [None]*n_datasets

for d_i in range(n_datasets):
    # Get path to dataset
    dataset_path = (Path(ps['dataset_base_folder']) / data_locs['Main folder'][d_i] / data_locs['Subfolder'][d_i] / 
                    Path(ps['dataset_folder']) / '*.pkl')
    dataset_file = glob.glob(str(dataset_path))
    if len(dataset_file) != 0:
        dataset_file = dataset_file[0]

        # Load the dataset
        with open(dataset_file, 'rb') as f:
            dataset = ROIDataset.from_dict(pickle.load(f))
    
        # Calculate dff
        f=dataset.ts_data[ps['f_ts_str']]['vls'][:]
        b=dataset.ts_data[ps['bl_ts_str']]['vls'][:]
        dff = calc_dff(f=f, b=b)
        
        # Extract dff before and after each manipulation event and store it 
        event_annot = dataset.metadata['manip_event_annotations']
        n_events = len(event_annot)
        dff_before = [None]*n_events
        dff_after = [None]*n_events
        for e_i in range(n_events):
            m_start = event_annot['Manipulation Start'].to_numpy()[e_i]
            m_end = event_annot['Manipulation End'].to_numpy()[e_i]
            dff_before[e_i] = np.mean(dff[m_start-3:m_start,:], axis=0)
            dff_after[e_i] = np.mean(dff[m_end+1:m_end+4,:], axis=0)

        event_annot['dff_before'] = dff_before
        event_annot['dff_after'] = dff_after
    
        # Save results
        event_annots[d_i] = event_annot
    
        # Give user some feedback
        print('Done processing dataset ' + str(d_i + 1) + ' of ' + str(n_datasets) + '.')

# Place event annotations into a single table
event_annots = pd.concat(event_annots)

min b: 48.071285
Done processing dataset 1 of 66.
min b: 102.64291
Done processing dataset 2 of 66.
min b: 101.983955
Done processing dataset 3 of 66.
min b: 89.34743
Done processing dataset 4 of 66.
min b: 4.5419083
Done processing dataset 5 of 66.
min b: 91.72084
Done processing dataset 6 of 66.
min b: 106.12106
Done processing dataset 7 of 66.
min b: 26.00942
Done processing dataset 8 of 66.
min b: 101.25182
Done processing dataset 9 of 66.
min b: 89.86631
Done processing dataset 10 of 66.
min b: 42.518364
Done processing dataset 11 of 66.
min b: 91.78164
Done processing dataset 12 of 66.
min b: 84.09773
Done processing dataset 13 of 66.
min b: 74.68948
Done processing dataset 14 of 66.
min b: 45.780064
Done processing dataset 15 of 66.
min b: 98.6193
Done processing dataset 16 of 66.
min b: 84.07614
Done processing dataset 17 of 66.
min b: 104.29418
Done processing dataset 18 of 66.
min b: 0.0
Done processing dataset 19 of 66.
min b: 0.0
Done processing dataset 21 of 66.
min b: 0.0

In [7]:
## Save results
rs = dict()
rs['ps'] = ps
rs['event_annots'] = event_annots

save_name = append_ts(ps['save_str']) + '.pkl'
save_path = Path(ps['save_folder']) / save_name
with open(save_path, 'wb') as f:
    pickle.dump(rs, f)
    
print('Saved results to: ' + str(save_path))

Saved results to: A:\projects\keller_vnc\results\whole_brain_stats\dff_5_25_25_with_ep_2019_10_29_15_22_01_205757.pkl


In [8]:
f=dataset.ts_data[ps['f_ts_str']]['vls'][:]
b=dataset.ts_data[ps['bl_ts_str']]['vls'][:]

In [10]:
np.min(b)

104.6817

In [19]:
(1 - 1/52)**52

0.36431351956896996